In [ ]:
from lib.ekyn import *
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from torch import nn
from torch.nn.functional import relu
from lib.models import *
from lib.deep_learning_utils import evaluate
from sklearn.metrics import ConfusionMatrixDisplay,classification_report,f1_score

ekyn_ids = get_ekyn_ids()
snezana_mice_ids = get_snezana_mice_ids()

n_samples = len(ekyn_ids) * 17280
n_seconds = n_samples * 10
n_hours = n_seconds / 3600
print(f'{n_hours} hours')

In [ ]:
model = ResNetv2(block=ResBlockv2,widthi=[4,8,16,32],depthi=[2,2,2,2],n_output_neurons=3)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=3e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)

In [ ]:
import numpy as np
ekyn_ids = np.array(ekyn_ids)
snezana_mice_ids = np.array(snezana_mice_ids)

In [ ]:
train_ids = {'ekyn':ekyn_ids[[0,1,2,3,4,5,6,7,8,9]],'snezana_mice':snezana_mice_ids[[0,1,2,3,4]]}
test_ids  = {'ekyn':ekyn_ids[[10,11,12,13,14,15]],'snezana_mice':snezana_mice_ids[[5,6,7,8]]}

from torch.utils.data import DataLoader,ConcatDataset
trainloader = DataLoader(
        dataset=ConcatDataset(
        [EpochedDataset(id=id,condition=condition,robust=True,downsampled=True) for id in train_ids['ekyn'] for condition in CONDITIONS] 
        + [EpochedDataset(id=id,snezana_mice=True) for id in train_ids['snezana_mice']]
        ),
        batch_size=1024,
        shuffle=True,
        num_workers=0
    )

testloader = DataLoader(
        dataset=ConcatDataset(
        [EpochedDataset(id=id,condition=condition,robust=True,downsampled=True) for id in test_ids['ekyn'] for condition in CONDITIONS] 
        + [EpochedDataset(id=id,snezana_mice=True) for id in test_ids['snezana_mice']]
        ),
        batch_size=1024,
        shuffle=True,
        num_workers=0
    )
print(train_ids,test_ids)
print(len(trainloader)*1024,len(testloader)*1024)

In [ ]:
trainlossi = []
testlossi = []

In [ ]:
ibatch = 0
nbatch = 10000
model.to('cuda')
from tqdm import tqdm
for i in tqdm(range(100)):
    model.train()
    for Xi,yi in trainloader:
        ibatch += 1
        Xi,yi = Xi.to('cuda'),yi.to('cuda')
        logits = model(Xi)
        loss = criterion(logits,yi)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        trainlossi.append(loss.item())

    model.eval()
    with torch.no_grad():
        loss_total = 0
        for Xi,yi in testloader:
            Xi,yi = Xi.to('cuda'),yi.to('cuda')
            logits = model(Xi)
            loss = criterion(logits,yi)
            loss_total += loss.item()
        testlossi.append(loss_total/len(testloader))
    
    plt.figure()
    plt.plot(torch.linspace(0,len(testlossi),len(trainlossi)),trainlossi)
    plt.plot(torch.linspace(0,len(testlossi),len(testlossi) + 1),[trainlossi[0]] + testlossi)
    plt.savefig('loss.jpg')
    plt.close()

    if ibatch >= nbatch:
        break

In [ ]:
fig,axes = plt.subplots(nrows=1,ncols=2,figsize=(8,4))
loss,y_true,y_pred = evaluate(dataloader=trainloader,model=model,criterion=criterion,device='cuda')
print(loss)
ConfusionMatrixDisplay.from_predictions(y_true,y_pred,normalize='true',ax=axes[0],colorbar=False)
axes[0].set_title(f'trainf1 : {f1_score(y_true,y_pred,average="macro"):.3f}')
loss,y_true,y_pred = evaluate(dataloader=testloader,model=model,criterion=criterion,device='cuda')
print(loss)
ConfusionMatrixDisplay.from_predictions(y_true,y_pred,normalize='true',ax=axes[1],colorbar=False)
axes[1].set_title(f'testf1 : {f1_score(y_true,y_pred,average="macro"):.3f}')

In [ ]:


df = pd.DataFrame()

plt.figure(figsize=(14,8))
for id in ekyn_ids[:10]:
    print(id)
    X,y = load_ekyn_pt_robust(id=id,condition='Vehicle')
    std,mean = torch.std_mean(X[torch.where(y.argmax(axis=1) == 1)[0]]
,dim=1)
    sns.ecdfplot(std,linewidth=2,label=id,color='blue')

# df_long = df.melt()
# df_long[['id','condition']] = df_long.variable.str.split('_',expand=True)
# sns.ecdfplot(df_long[df_long.condition == 'Vehicle'],x='value',hue='id',linestyle='-')
# sns.ecdfplot(df_long[df_long.condition == 'PF'],x='value',hue='id',linestyle='--')
# std,mean = torch.std_mean(eeg,dim=1)

plt.legend()
plt.xlim([0,2])

In [ ]:
X.shape

In [ ]:
df = pd.DataFrame()

plt.figure(figsize=(14,8))
for id in idx:
    for condition in ['Vehicle','PF']:
        X,y = load_ekyn_pt(idx=id,condition=condition)
        std,mean = torch.std_mean(X,dim=1)
        df[f'{id}_{condition}'] = std
df_long = df.melt()
df_long[['id','condition']] = df_long.variable.str.split('_',expand=True)
sns.ecdfplot(df_long[df_long.condition == 'Vehicle'],x='value',hue='id',linestyle='-')
sns.ecdfplot(df_long[df_long.condition == 'PF'],x='value',hue='id',linestyle='--')
std,mean = torch.std_mean(eeg,dim=1)
sns.ecdfplot(std,linewidth=2)


plt.xlim([0,.0002])

In [ ]:
idx = get_ekyn_ids()
# train_idx,test_idx = train_test_split(get_ekyn_ids(),test_size=.25,random_state=0)
train_idx = get_ekyn_ids()
test_idx = [train_idx[12]]
train_idx.remove(test_idx[0])
print(train_idx,test_idx)
print(len(train_idx),len(test_idx))

In [ ]:
df = pd.DataFrame()

for id in idx:
    for condition in ['Vehicle','PF']:
        X,y = load_ekyn_pt(idx=id,condition=condition)
        df[f'{id}_{condition}'] = torch.bincount(y.argmax(axis=1))/len(y)

In [ ]:
df = df.T

In [ ]:
plt.figure(figsize=(7.2,7.2),dpi=200)
sns.boxplot(data=df)

In [ ]:
s = df.describe()[1]
w = df.describe()[2]
iqr_s = s['75%'] - s['25%']
iqr_w = w['75%'] - w['25%']
lower_outlier_s = s['25%'] - (1.5*iqr_s)
upper_outlier_w = w['75%'] + (1.5*iqr_w)
lower_outlier_s

In [ ]:
df[(df[1] < lower_outlier_s) | (df[2] > upper_outlier_w)]

In [ ]:
from mne.io import read_raw_edf

eeg = torch.from_numpy(read_raw_edf(f'./24-AGING-2.edf').get_data(picks=['EEG 1'])[0]).float().reshape(-1,5000)

In [ ]:
df = pd.DataFrame()

plt.figure(figsize=(14,8))
for id in idx:
    for condition in ['Vehicle','PF']:
        X,y = load_ekyn_pt(idx=id,condition=condition)
        std,mean = torch.std_mean(X,dim=1)
        df[f'{id}_{condition}'] = std
df_long = df.melt()
df_long[['id','condition']] = df_long.variable.str.split('_',expand=True)
sns.ecdfplot(df_long[df_long.condition == 'Vehicle'],x='value',hue='id',linestyle='-')
sns.ecdfplot(df_long[df_long.condition == 'PF'],x='value',hue='id',linestyle='--')
std,mean = torch.std_mean(eeg,dim=1)
sns.ecdfplot(std,linewidth=2)


plt.xlim([0,.0002])